In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 1.x

In [ ]:
!pip install tiffile
!pip install csbdeep
!pip install keras==2.2.5
!pip install cdifflib
!pip install scikit-image

In [ ]:
%cd '/content/drive/My Drive/'
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from csbdeep.utils import Path, normalize
import numpy as np
import matplotlib.pyplot as plt
import glob
from tifffile import imread, imwrite
from skimage.morphology import remove_small_objects
from csbdeep.utils import Path, normalize
from csbdeep.models import Config, CARE
import pandas as pd
import difflib
from matplotlib import cm
from skimage.filters import threshold_otsu
import numpy as np
import skimage
from skimage import transform
from scipy.ndimage.morphology import binary_fill_holes
from pathlib import Path
from skimage.filters import sobel
from skimage.util import invert
from skimage.measure import label
from PIL import Image

In [ ]:
Masterdir = '/content/drive/My Drive/R/'
Savedir = '/content/drive/My Drive/UnRotated/'
Model_Dir = '/content/drive/My Drive/SegmentationModel/'
UNETSegmentationModelName = 'DeepWingSegmentationModelUNET'

UnetModel = CARE(config = None, name = UNETSegmentationModelName, basedir = Model_Dir)
Path(Savedir).mkdir(exist_ok = True)

LeftName = 'L'
RightName = 'R'
show_after = 100
MaskResults = Savedir + '/MaskResults/'
AsymmetryResults = Savedir + '/AsymmetryResults/'
AsymmetryResultsName = 'Asymmetry'
Path(MaskResults).mkdir(exist_ok = True)
Path(AsymmetryResults).mkdir(exist_ok = True)

**Hello, You only have to change one thing in this code which is the Masterdir, here you specify the path to the directory where your images are. After that you can do Cell Run all/ Kernel restart run all from the menu above to run the code**

In [ ]:
def doubleplot(imageA, imageB, titleA, titleB, targetdir = None, File = None, plotTitle = None):
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    ax = axes.ravel()
    ax[0].imshow(imageA, cmap=cm.gray)
    ax[0].set_title(titleA)
    ax[0].set_axis_off()
    ax[1].imshow(imageB, cmap=plt.cm.nipy_spectral)
    ax[1].set_title(titleB)
    ax[1].set_axis_off()

    plt.tight_layout()
    plt.show()
    for a in ax:
      a.set_axis_off() 

In [ ]:
Raw_path = os.path.join(Masterdir, '*tiff')
filesRaw = glob.glob(Raw_path)
filesRaw.sort
axes = 'YXC'
count = 0
min_size = 15000
for fname in filesRaw:
            
           #Read image  
            Name = os.path.basename(os.path.splitext(fname)[0])
            
            if Name not in Done:
                try:
                    image = imread(fname)


                    image = image[:,:,0:3]
                    #DO the segmentation
                    Segmented = UnetModel.predict(image,axes)
                    thresh = threshold_otsu(Segmented) 
                    Binary = Segmented > thresh
                    Filled = binary_fill_holes(Binary)
                    Finalimage = remove_small_objects(Filled, min_size)
                    Finalimage = Finalimage[:,:,0]
                    y, x = np.nonzero(Finalimage)
                    x = x - np.mean(x)
                    y = y - np.mean(y)
                    coords = np.vstack([x, y])

                    cov = np.cov(coords)
                    evals, evecs = np.linalg.eig(cov) 


                    sort_indices = np.argsort(evals)[::-1]
                    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
                    x_v2, y_v2 = evecs[:, sort_indices[1]]

                    #Uncomment lines below to see the eigenvectors
                    #scale = 20
                    #plt.plot([x_v1*-scale*2, x_v1*scale*2],
                            #[y_v1*-scale*2, y_v1*scale*2], color='red')
                    #plt.plot([x_v2*-scale, x_v2*scale],
                            #[y_v2*-scale, y_v2*scale], color='blue')

                    #plt.axis('equal')
                    #plt.gca().invert_yaxis()  # Match the image system with origin at top left
                    #plt.show()
                    theta1 = np.arctan((x_v1)/(y_v1)) 
                    theta2 = np.arctan((x_v2)/(y_v2)) 
                    theta2deg = theta2 * 180 / 3.14
                    theta1deg = theta1 * 180/  3.14


                    rotation_mat = np.matrix([[np.cos(theta2), -np.sin(theta2)],
                              [np.sin(theta2), np.cos(theta2)]])
                    rotatedimage = transform.rotate(image,-theta2deg, resize = False, mode = "edge" )


                    #Trial 2
                    #DO bad segmentation

                    testimage = rotatedimage[:,:,0] 
                    thresh = threshold_otsu(testimage) 
                    testimage = testimage > thresh
                    testimage = invert(testimage)
                    testimage = label(testimage)

                    testimage = remove_small_objects(testimage, min_size)
                    testimage = testimage > 0



                    ySec, xSec = np.nonzero(testimage)
                    xSec = xSec - np.mean(xSec)
                    ySec = ySec - np.mean(ySec)
                    coordsSec = np.vstack([xSec, ySec])

                    try:
                        covSec = np.cov(coordsSec)
                        evalsSec, evecsSec = np.linalg.eig(covSec) 


                        sort_indicesSec = np.argsort(evalsSec)[::-1]
                        x_v1Sec, y_v1Sec = evecsSec[:, sort_indicesSec[0]]  # Eigenvector with largest eigenvalue
                        x_v2Sec, y_v2Sec = evecsSec[:, sort_indicesSec[1]]


                        theta1Sec = np.arctan((x_v1Sec)/(y_v1Sec)) 
                        theta2Sec = np.arctan((x_v2Sec)/(y_v2Sec)) 
                        theta2degSec = theta2Sec * 180 / 3.14
                        theta1degSec = theta1Sec * 180/3.14
                    except: 
                            theta2degSec = 0
                            theta1degSec = 0

                    count = count + 1
                    if Name[-1] == 'R'and theta1degSec < 0:
                        flippedimage = np.flip(rotatedimage, axis = 0)
                        imwrite(Savedir + Name + '.tif', flippedimage)
                        if count%show_after == 0:
                           doubleplot(image, flippedimage, "Original", "Rotated-and-Flipped")

                    if Name[-1] == 'R'and theta1degSec >= 0:   
                        imwrite(Savedir + Name + '.tif', rotatedimage)
                        if count%show_after == 0:
                            doubleplot(image, rotatedimage, "Original", "Rotated")    

                    if Name[-1] == 'L'and theta1degSec >= 0:
                        flippedimage = np.flip(rotatedimage, axis = 0)
                        imwrite(Savedir + Name + '.tif', flippedimage)
                        if count%show_after == 0:
                           doubleplot(image, flippedimage, "Original", "Rotated-and-Flipped")    

                    if Name[-1] == 'L'and theta1degSec < 0:   
                        imwrite(Savedir + Name + '.tif', rotatedimage)
                        if count%show_after == 0:
                            doubleplot(image, rotatedimage, "Original", "Rotated")
                except:
                   print(Name) 
                   pass


In [ ]:
#Apply the prediction on wing directory

Raw_path = os.path.join(Savedir, '*tif')

axes = 'YXC'
saveaxes = 'YX'
filesRaw = glob.glob(Raw_path)
filesRaw.sort
count = 0
axis_norm = (0,1,2)   # normalize channels independently

for fname in filesRaw:
            
            #Read image        
            image = imread(fname)
            
            originalX = image.shape[0]
            originalY = image.shape[1]  
            
            Name = os.path.basename(os.path.splitext(fname)[0])
      
          
            if Name[-1] == 'L':
                image = transform.rotate(image,  180,  resize=False)
                image = np.flip(image, axis = 0)
            x = image[:,:,0:3]
            

            #Make sure image is 2D

            Segmented = UnetModel.predict(x,axes)
            thresh = threshold_otsu(Segmented) 
            Binary = Segmented > thresh
            Filled = binary_fill_holes(Binary)
            Finalimage = remove_small_objects(Filled, min_size)
     
            
            
            if count%show_after == 0:
              doubleplot(image,Finalimage[:,:,0], 'Original', 'UNET', plotTitle = 'Segmentation Result' )
            count = count + 1 
            imwrite((MaskResults + 'Mask' + Name + '.tif' ) , Finalimage.astype('uint8'))


In [ ]:
import pandas as pd
Path(AsymmetryResults).mkdir(exist_ok = True)
Raw_pathRight = os.path.join(MaskResults, '*tif')
Raw_pathLeft = os.path.join(MaskResults, '*tif')

filesRawRight = glob.glob(Raw_pathRight)
filesRawLeft = glob.glob(Raw_pathLeft)


AllRightArea = []
AllLeftArea = []
AllRightMinusLeftArea = []
AllRightPlusLeftArea = []
AllAssymetery = []
AllName = []
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
for fnameRight in filesRawRight:
    
   NameRight = os.path.basename(os.path.splitext(fnameRight)[0]) 
   imageRight = imread(fnameRight)
   for fnameLeft in filesRawLeft:
       NameLeft = os.path.basename(os.path.splitext(fnameLeft)[0]) 
       imageLeft = imread(fnameLeft) 
       
       ChangeName = difflib.ndiff(NameLeft, NameRight)
       delta = ''.join(x[0:] for x in ChangeName if x.startswith('- '))
       
       #ChangeName = NameRight.replace(RightName, LeftName) 
       if delta == '- L':
           print(NameLeft, NameRight)
           RightArea, LeftArea, RightMinusLeft, RightPlusLeft, Assymetery = WingArea(imageLeft, imageRight)
         
           AllName.append(NameLeft)
           AllRightArea.append(RightArea)
           AllLeftArea.append(LeftArea)
           AllRightMinusLeftArea.append(RightMinusLeft)
           AllRightPlusLeftArea.append(RightPlusLeft)
           AllAssymetery.append(Assymetery)
            
df = pd.DataFrame(list(zip(AllRightArea,AllLeftArea,AllRightMinusLeftArea,AllRightPlusLeftArea,AllAssymetery)), index =AllName, 
                                              columns =['RightArea', 'LeftArea', 'Right-Left', 'Right+Left', 'Assymmetery'])

df.to_csv(AsymmetryResults + '/' + AsymmetryResultsName +  '.csv')  
df

In [ ]:
positivecount = np.sum(df['Assymmetery']>0)
negativecount = np.sum(df['Assymmetery']<0)   
print('Positive Count' , positivecount)
print('Negative Count' , negativecount)

In [ ]:
plt.plot(AllAssymetery)
plt.title("Asymmetry")
plt.ylabel("Asymmetry")
plt.xlabel("Filenumber")
plt.show()